In [1]:
import numpy as np
import os
import sys
import time
import argparse
import json
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

import torch
import torch.utils.data as data
import torch.optim as optim
from torch.autograd import Variable

sys.path.insert(0,'../modules')
from sample_generator import *
from data_prov import *
from model import *
from bbreg import *
from options import *
from gen_config import *

In [2]:
np.random.seed(123)
torch.manual_seed(456)
torch.cuda.manual_seed(789)

In [3]:
seq_home = '../dataset/OTB'
save_home = '../result_fig'
result_home = '../result'

seq_name = 'angle_3'
img_dir = os.path.join(seq_home, seq_name, 'img')
gt_path = os.path.join(seq_home, seq_name, 'dec.txt')

img_list = os.listdir(img_dir)
img_list.sort()
img_list = [os.path.join(img_dir,x) for x in img_list]

gt = np.loadtxt(gt_path,delimiter=',')
init_bbox = gt[gt[:,1] == 43, :]
init_bbox_sc = init_bbox[:,-1]
init_bbox = init_bbox[np.argsort(-init_bbox_sc), :]
print init_bbox
for i in range(init_bbox.shape[0] - 1):
    if init_bbox[i,0] != -1:
        for j in range(i + 1, init_bbox.shape[0]):
            if(overlap_ratio(init_bbox[i,2:6], init_bbox[j,2:6]) >= 0.25):
                init_bbox[j,0] = -1
init_bbox = init_bbox[init_bbox[:,0] != -1, :]

[[  7.00000000e+00   4.30000000e+01   8.54374000e+02   6.17332000e+02
    1.41502563e+02   1.12371704e+02   9.99744000e-01]
 [  7.00000000e+00   4.30000000e+01   1.03730000e+03   6.08762000e+02
    1.47087524e+02   1.45620117e+02   9.96687000e-01]
 [  7.00000000e+00   4.30000000e+01   9.81984000e+02   4.44078000e+02
    8.20979004e+01   6.80032654e+01   9.42352000e-01]
 [  7.00000000e+00   4.30000000e+01   4.83548000e+02   3.84923000e+02
    2.62966736e+02   1.52258240e+02   7.45481000e-01]
 [  7.00000000e+00   4.30000000e+01   5.99469000e+02   3.79933000e+02
    1.50129761e+02   1.19243286e+02   5.72236000e-01]
 [  6.00000000e+00   4.30000000e+01   4.87037000e+02   3.87754000e+02
    2.64858490e+02   1.44066284e+02   3.18454000e-01]
 [  7.00000000e+00   4.30000000e+01   1.12542000e+03   4.54481000e+02
    6.35576172e+01   4.85520630e+01   3.01737000e-01]]


In [4]:
def multidim_intersect(arr1, arr2):
    arr1_view = arr1.view([('',arr1.dtype)]*arr1.shape[1])
    arr2_view = arr2.view([('',arr2.dtype)]*arr2.shape[1])
    print (arr1_view.shape, arr2_view.shape)
    print (arr1_view, arr2_view)
    intersected = np.intersect1d(arr1_view, arr2_view)
    return intersected.view(arr1.dtype).reshape(-1, arr1.shape[1])

In [7]:
test_arr1 = np.array([[0., 2., 3., 4.],
                         [1., 3., 3., 4.],
                         [4., 5., 2., 1.],
                         [0., 2., 2., 1.]])

test_arr2 = np.array([[1., 2., 2., 1.],
                         [0., 2., 3, 4.],
                         [1., 3., 3, 4.]])
print (test_arr2.shape, test_arr1.shape)

print multidim_intersect(test_arr1, test_arr2)

((3, 4), (4, 4))
((4, 1), (3, 1))
(array([[( 0.,  2.,  3.,  4.)],
       [( 1.,  3.,  3.,  4.)],
       [( 4.,  5.,  2.,  1.)],
       [( 0.,  2.,  2.,  1.)]],
      dtype=[('f0', '<f8'), ('f1', '<f8'), ('f2', '<f8'), ('f3', '<f8')]), array([[( 1.,  2.,  2.,  1.)],
       [( 0.,  2.,  3.,  4.)],
       [( 1.,  3.,  3.,  4.)]],
      dtype=[('f0', '<f8'), ('f1', '<f8'), ('f2', '<f8'), ('f3', '<f8')]))
[[ 0.  2.  3.  4.]
 [ 1.  3.  3.  4.]]
